In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [2]:
data = pd.read_csv(r"D:\ML_Projects\VS_project\Churn_Modelling.csv")
data.head() #clearly 1st 3 features not very important

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data = data.drop(['RowNumber','CustomerId','Surname'], axis = 1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
#Encode categorical variables
label_encode_gender = LabelEncoder()
data['Gender'] = label_encode_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [5]:
#do onehotencoding for Geography column to prevent hierarchy 
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder_geo = OneHotEncoder(sparse_output=False)
geo_encoder = one_hot_encoder_geo.fit_transform(data[['Geography']])
print(one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoder

['Geography_France' 'Geography_Germany' 'Geography_Spain']


array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [6]:
#By default, one-hot encoding returns a sparse matrix, which saves memory by only storing non-zero values. When sparse_output=False is set, the encoder will instead produce a dense NumPy array as output. This can be more memory-intensive but easier to work with for smaller datasets or when sparse matrices are unnecessary.

In [7]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [8]:
geo_encoded_df = pd.DataFrame(geo_encoder, columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [9]:
#Concatenate this dataframe with our original data
data = pd.concat([data.drop('Geography', axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
#Save the encoders & scaler
with open('label_encode_gender.pkl','wb') as file:
    pickle.dump(label_encode_gender,file)
with open('one_hot_encoder_geo.pkl','wb') as file:
    pickle.dump(one_hot_encoder_geo,file)

In [11]:
#Divide dataset into dependent & indep features
X = data.drop('Exited',axis=1)
y = data['Exited']

#Split data in train & test sets
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

#Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
#Also save these files in pkl format
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

!pip install tensorflow

In [14]:
import tensorflow as tf
print(tf.__path__)

['C:\\Users\\Aero\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\keras\\api\\_v2', 'C:\\Users\\Aero\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\keras\\_tf_keras', 'C:\\Users\\Aero\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\tensorflow', 'C:\\Users\\Aero\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\tensorflow\\_api\\v2']


In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
from tensorflow.keras.layers import Dense, Input

In [16]:
(X_train.shape[1],) 
#The comma in (X_train.shape[1],) indicates that it’s a tuple with a single element. This ensures the shape is treated as a tuple rather than a simple integer, which some functions or models require.

(12,)

In [17]:
#Build ANN model, always for the first hidden layer we need to write input_shape, not for further hidden layers
model = Sequential([
    Input(shape=(X_train.shape[1],)),   # Define the input layer shape
    Dense(64, activation='relu'),       # HL1
    Dense(32, activation='relu'),       # HL2
    Dense(1, activation='sigmoid')      # Output layer
])


In [18]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [20]:
#Compile the model
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])#If I dirctly provide adam here, it'll have a fixed learning rate thus use opt

In [21]:
#Set up the tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir = 'logs/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [22]:
#Set up Early Stopping if good accuracy reached at some epoch
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True) #validation loss

In [23]:
#Train the model
history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.8121 - loss: 0.4432 - val_accuracy: 0.8545 - val_loss: 0.3466
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8578 - loss: 0.3557 - val_accuracy: 0.8545 - val_loss: 0.3536
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8546 - loss: 0.3577 - val_accuracy: 0.8640 - val_loss: 0.3467
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8571 - loss: 0.3445 - val_accuracy: 0.8470 - val_loss: 0.3529
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8652 - loss: 0.3345 - val_accuracy: 0.8615 - val_loss: 0.3524
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8599 - loss: 0.3467 - val_accuracy: 0.8550 - val_loss: 0.3658
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8604 - loss: 0.3472 - val_accuracy: 0.8555 - val_loss: 0.3725
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8661 - loss: 0.3289 - val_accu

In [24]:
model.save('model.h5')

In [25]:
#Load Tensorboard Extension
%load_ext tensorboard

In [28]:
tensorboard --logdir=logs/fit --port=6014

In [27]:
#For deployment of app.py use streamlit run app.py on command prompt directly